<a href="https://colab.research.google.com/github/viktorngkhnh/BearingData/blob/main/CNN_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Mount Drive
drive.mount('/content/drive')

DATA_PATH = '/content/drive/My Drive/Bearing/DE_data'

if os.path.exists(DATA_PATH):
    print(f"{DATA_PATH}")
    print("Fld:", os.listdir(DATA_PATH))

In [ ]:
# --- CELL 2: VISUALIZE RAW DATA (Soi dữ liệu thô) ---

def plot_raw_sample(root_dir):
    #Define
    target_folders = ["Normal", "Ball_007", "Ball_014", "Ball_021", "Ball_028"]

    plt.figure(figsize=(15, 10))
    plt.subplots_adjust(hspace=0.4)

    found_count = 0

    # Check fld
    if os.path.exists(root_dir):
        all_folders = os.listdir(root_dir)

        for i, target in enumerate(target_folders):

            folder_name = next((f for f in all_folders if target in f), None)

            if folder_name:
                folder_path = os.path.join(root_dir, folder_name)
                # Get .mat
                files = [f for f in os.listdir(folder_path) if f.endswith('.mat')]

                if files:
                    file_path = os.path.join(folder_path, files[0])
                    try:

                        mat = scipy.io.loadmat(file_path)
                        key = [k for k in mat.keys() if 'DE_time' in k][0]
                        signal = mat[key].flatten()

                        #plt.subplot(2, 2, i + 1)
                        plt.figure(figsize=(15, 5))
                        plt.ylim(-2 , 2)
                        plt.plot(signal[:12000])
                        plt.title(f"{target} (File: {files[0]})")
                        plt.ylabel("Amplitude")
                        plt.grid(True)
                        found_count += 1
                    except Exception as e:
                        print(f"Lỗi đọc file {files[0]}: {e}")
            else:
                print(f"⚠️ Không thấy folder nào chứa tên '{target}'")

    if found_count == 4:
        print("✅ Đã vẽ đủ 4 loại tín hiệu. Hãy quan sát hình bên dưới!")
    else:
        print(f"⚠️ Chỉ tìm thấy {found_count}/4 loại dữ liệu.")
    plt.show()


plot_raw_sample(DATA_PATH)

In [ ]:
def transform_1d_to_2d(signal, img_size=32):
    """
    Cắt tín hiệu 1D thành các đoạn và reshape thành ma trận vuông (ảnh xám).
    Ví dụ: 1024 mẫu -> (32, 32)
    """
    window_size = 2 * img_size * img_size  # 32 * 32 * 2  = 2048

    # 1. Tính số lượng ảnh có thể tạo ra (bỏ phần dư cuối cùng)
    num_images = len(signal) // window_size

    # 2. Cắt bỏ phần dư để đảm bảo chia hết cho 1024
    truncated_signal = signal[:num_images * window_size]

    # 3. Reshape sang (Số lượng ảnh, 32, 32)
    images_2d = truncated_signal.reshape(-1, img_size, img_size)

    # 4. Thêm một chiều (Channel) để PyTorch/CNN hiểu là ảnh xám (N, 1, 32, 32)
    images_2d = np.expand_dims(images_2d, axis=1)

    return images_2d


In [ ]:
from scipy import signal

def plot_spectrograms(root_dir):
    target_folders = ["Normal", "Ball_007", "Ball_014", "Ball_021"]
    plt.figure(figsize=(18, 12))

    found_count = 0
    all_folders = os.listdir(root_dir)

    for i, target in enumerate(target_folders):
        folder_name = next((f for f in all_folders if target in f), None)
        if folder_name:
            folder_path = os.path.join(root_dir, folder_name)
            files = [f for f in os.listdir(folder_path) if f.endswith('.mat')]

            if files:
                file_path = os.path.join(folder_path, files[0])
                mat = scipy.io.loadmat(file_path)
                key = [k for k in mat.keys() if 'DE_time' in k][0]
                sig = mat[key].flatten()[:120000] # Lấy một đoạn ngắn để vẽ

                # --- BƯỚC QUAN TRỌNG: TẠO SPECTROGRAM ---
                # fs: tần số lấy mẫu (CWRU thường là 12000 hoặc 48000 Hz)
                fs = 12000
                frequencies, times, Sxx = signal.spectrogram(sig, fs=fs, nperseg=512)

                plt.subplot(2, 2, i + 1)
                # Dùng pcolormesh để vẽ ma trận 2D
                plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx), shading='gouraud', cmap='jet')
                plt.title(f"Spectrogram: {target}")
                plt.ylabel("Frequency [Hz]")
                plt.xlabel("Time [sec]")
                plt.colorbar(label="Intensity [dB]")
                found_count += 1

    plt.tight_layout()
    plt.show()

# Gọi hàm để xem kết quả
plot_spectrograms(DATA_PATH)

In [ ]:
import cv2 # Thư viện xử lý ảnh

def get_spectrogram_image(sig, fs=12000, img_size=64):
    # 1. Tạo Spectrogram thô
    freqs, times, Sxx = signal.spectrogram(sig, fs=fs, nperseg=256)

    # 2. Chuyển sang thang đo Log để nổi bật đặc trưng
    Sxx_log = 10 * np.log10(Sxx + 1e-10) # Thêm số nhỏ để tránh lỗi log(0)

    # 3. Chuẩn hóa về khoảng [0, 255] để thành ảnh xám chuẩn
    Sxx_norm = cv2.normalize(Sxx_log, None, 0, 255, cv2.NORM_MINMAX)
    Sxx_norm = Sxx_norm.astype(np.uint8)

    # 4. Resize về kích thước cố định (ví dụ 64x64)
    img_resized = cv2.resize(Sxx_norm, (img_size, img_size))

    return img_resized

# --- THỰC HÀNH THỬ ---
# img = get_spectrogram_image(signal)
# plt.imshow(img, cmap='jet')
# print(f"Kích thước ảnh mới: {img.shape}")

In [ ]:
class BearingSpectrogramDataset(Dataset):
    def __init__(self, root_dir, window_size=2048, stride=512, img_size=64):
        self.samples = []
        self.labels = []
        self.img_size = img_size

        target_folders = ["Normal", "Ball_007", "Ball_014", "Ball_021", "Ball_028"]
        label_map = {name: i for i, name in enumerate(target_folders)}

        all_folders = os.listdir(root_dir)
        for folder in all_folders:
            match = next((t for t in target_folders if t in folder), None)
            if match:
                label = label_map[match]
                folder_path = os.path.join(root_dir, folder)
                files = [f for f in os.listdir(folder_path) if f.endswith('.mat')]

                for file in files:
                    file_path = os.path.join(folder_path, file)
                    mat = scipy.io.loadmat(file_path)

                    key = [k for k in mat.keys() if 'time' in k and ('DE' in k or 'FE' in k)][0]
                    signal_full = mat[key].flatten()

                    # KỸ THUẬT OVERLAPPING:
                    # Thay vì nhảy bước bằng window_size, ta nhảy bước bằng stride (nhỏ hơn)
                    # Nếu window=2048 và stride=512, ta sẽ có gấp 4 lần dữ liệu
                    idx = 0
                    count_per_file = 0
                    while idx + window_size <= len(signal_full):
                        segment = signal_full[idx : idx + window_size]

                        # Tạo Spectrogram
                        f, t, Sxx = signal.spectrogram(segment, fs=12000)
                        Sxx_log = 10 * np.log10(Sxx + 1e-10)
                        Sxx_norm = cv2.normalize(Sxx_log, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                        img = cv2.resize(Sxx_norm, (img_size, img_size))

                        self.samples.append(img)
                        self.labels.append(label)

                        idx += stride # Nhảy một bước ngắn
                        count_per_file += 1

                        # Giới hạn để tránh quá tải RAM nếu cần (ví dụ 1000 ảnh/file)
                        if count_per_file >= 1000: break

        self.samples = np.array(self.samples)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img = torch.tensor(self.samples[idx], dtype=torch.float32).unsqueeze(0) / 255.0
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return img, label

# Khởi tạo lại với stride=512 (Overlap 75%)
dataset_overlap = BearingSpectrogramDataset(DATA_PATH, window_size=2048, stride=512)
train_loader = DataLoader(dataset_overlap, batch_size=32, shuffle=True)

print(f"🚀 Số lượng ảnh sau khi Overlap: {len(dataset_overlap)}")

In [ ]:
from collections import Counter
label_counts = Counter(dataset.labels)
target_folders = ["Normal", "Ball_007", "Ball_014", "Ball_021", "Ball_028"]

print("Phân bổ dữ liệu:")
for i, name in enumerate(target_folders):
    print(f"- {name}: {label_counts[i]} ảnh")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class BearingCNN(nn.Module):
    def __init__(self):
        super(BearingCNN, self).__init__()
        # Convolutional Layer 1: In Gray layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # Convolutional Layer 2: In 16channels Out 32chanels
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)

        # Pooling 2times 64x64 to 16x16
        # Flatten layer: 32 channels * 16 * 16 = 8192 nodes
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BearingCNN().to(device)

print(f"✅ Đã định nghĩa và khởi tạo 'model' trên {device} thành công!")

In [ ]:
import time

# 1. Cấu hình các thông số
num_epochs = 15
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Chuyển mô hình sang GPU/CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"🚀 Bắt đầu huấn luyện 15 epochs trên thiết bị: {device}")
start_time = time.time()

for epoch in range(num_epochs):
    model.train() # Chuyển sang chế độ huấn luyện
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:

        inputs, labels = inputs.to(device), labels.to(device)


        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()


        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total

    print(f"Epoch [{epoch+1:02d}/{num_epochs}] - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.2f}%")

end_time = time.time()
print(f"\n✅ Hoàn thành huấn luyện trong: {(end_time - start_time)/60:.2f} phút")

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(model, loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=target_folders, yticklabels=target_folders)
    plt.xlabel('(Predicted)')
    plt.ylabel('(Actual)')
    plt.title('Confusion Matrix')
    plt.show()

plot_confusion_matrix(model, train_loader)

In [ ]:
# Thay đổi đường dẫn này tới thư mục FE của bạn
DATA_PATH_FE = '/content/drive/My Drive/Bearing/FE_data'

if os.path.exists(DATA_PATH_FE):
    print("✅ Đã tìm thấy dữ liệu FE để test!")
else:
    print("❌ Đường dẫn FE không đúng, hãy kiểm tra lại!")

In [ ]:
# Khởi tạo dataset cho FE
# Chúng ta vẫn dùng window_size và img_size giống như lúc train (2048 và 64)
test_dataset_fe = BearingSpectrogramDataset(DATA_PATH_FE, window_size=2048, img_size=64)
test_loader_fe = DataLoader(test_dataset_fe, batch_size=32, shuffle=False)

print(f"Tổng số ảnh Spectrogram tạo từ FE: {len(test_dataset_fe)}")

In [ ]:
model.eval() # Chuyển mô hình sang chế độ đánh giá (quan trọng!)
correct = 0
total = 0

all_preds = []
all_labels = []

with torch.no_grad(): # Không tính toán gradient để tiết kiệm RAM
    for inputs, labels in test_loader_fe:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy_fe = 100 * correct / total
print(f"📊 Độ chính xác (Accuracy) trên tập FE: {accuracy_fe:.2f}%")

In [ ]:
cm_fe = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_fe, annot=True, fmt='d', cmap='Oranges',
            xticklabels=target_folders, yticklabels=target_folders)
plt.title('Confusion Matrix - Testing on FE Data')
plt.show()